In [2]:
using Revise
using Pkg; Pkg.activate(".")
using HypergraphModularity
using StatsBase
using DataFrames
using RCall
using SparseArrays
using Clustering

 Activating environment at `~/hypergraph_modularities_code/Project.toml`
┌ Info: Precompiling HypergraphModularity [0c934d27-dd44-49d7-950f-bd4be7819e54]
└ @ Base loading.jl:1260
┌ Warning: Package HypergraphModularity does not have DelimitedFiles in its dependencies:
│ - If you have HypergraphModularity checked out for development and have
│   added DelimitedFiles as a dependency but haven't updated your primary
│   environment's manifest file, try `Pkg.resolve()`.
│ - Otherwise you may need to report an issue with HypergraphModularity
└ Loading DelimitedFiles into HypergraphModularity from project dependency, future warnings for HypergraphModularity are suppressed.
  ** incremental compilation may be fatally broken for this module **

  ** incremental compilation may be fatally broken for this module **



In [11]:
# H, Z = read_hypergraph_data("walmart-trips", 20)
kmax = 10
H, Z = read_hypergraph_data("SN-congress-bills", kmax) # VERY good results on Trivago with 3-5 core
# H, Z = read_hypergraph_data("congress-bills", 20) 

Base.copy(H::hypergraph) = hypergraph(H.N, H.E, H.D)

In [13]:
V = big(sum(H.D))
function ω_d(p, α)
    k = big(sum(p))
    kmax = length(α) ÷ 2
    dcut = big((sum(p*p') - p'*p)/2)
    return k/((1 + dcut)^α[k])/ V^(α[kmax+k]*k)
end
Ω̂ = partitionIntensityFunction(ω_d, kmax)
α = vcat(repeat([1.0], kmax), 1:kmax);
α = learnParameters(H, Z, Ω̂, α; max_iters = 50, verbose = false, tol = 1e-2) 

20-element Array{Float64,1}:
   1.0
   0.5091902583294604
   0.7324012598771031
   0.8718804537782678
   0.963534608267226
   0.9681938699578093
   1.1874155680088547
   1.245955991831588
   1.3162636324396662
   1.3993788395851705
 280.88472054342907
   0.662293897473341
   0.7784901770366114
   0.8351706524707061
   0.8698429227306783
   0.8912336338136182
   0.902125163200449
   0.9120316963364965
   0.9187967325061648
   0.9252202070017719

In [14]:
Ẑ = SuperNodeLouvain(H::hypergraph, Ω̂; α = α, verbose = true)

Faster SuperNode Louvain: Phase 1
Louvain Iteration 1, Q = -716461.98
Louvain Iteration 2, Q = -709819.72
Louvain Iteration 3, Q = -707206.63
Louvain Iteration 4, Q = -706883.36
Louvain Iteration 5, Q = -706627.17
SuperNode Louvain: Phase 2
toc = 0.06918716430664062
Louvain Iteration 1
Louvain Iteration 2
t1 = 0.598656415939331
SuperNode Louvain: Phase 3
toc = 0.012814998626708984
Louvain Iteration 1
t1 = 0.036128997802734375


294-element Array{Int64,1}:
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 1
 ⋮
 3
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2
 2

In [15]:
randindex(Z, Ẑ)[1]

0.045501929577378905

In [17]:
countmap(Ẑ)

Dict{Int64,Int64} with 3 entries:
  2 => 150
  3 => 1
  1 => 143